# Machine Perception Project 2


Members: Johnny Lin, Matt(Zitong) Wei, Boshen Pan

# Create directory

We will store the raw images/raw videos in this directory. If using colab, you will need to upload your videos/images after creating this directory.

In [ ]:
import os
custom_dir = "/content/data/nerfstudio/custom_data/raw_images"

if not(os.path.exists(custom_dir)):
  os.makedirs(custom_dir)

custom_vid = "/content/data/nerfstudio/custom_data/raw_video"

if not(os.path.exists(custom_vid)):
  os.makedirs(custom_vid)

# Install Nerfstudio and dependencies


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/
!pip install --upgrade pip
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

# Installing TinyCuda
%cd /content/
!gdown "https://drive.google.com/u/1/uc?id=1-7x7qQfB7bIw2zV4Lr6-yhvMpjXC84Q5&confirm=t"
!pip install tinycudann-1.7-cp310-cp310-linux_x86_64.whl

!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c conda-forge colmap

# Install nerfstudio
%cd /content/
!pip install git+https://github.com/nerfstudio-project/nerfstudio.git

/content
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 66.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 116.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 159.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 111.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 121.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 122.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11

Verify whether colmap is installed

In [ ]:
!colmap -h

COLMAP 3.11.1 -- Structure-from-Motion and Multi-View Stereo
(Commit Unknown on Unknown with CUDA)

Usage:
  colmap [command] [options]

Documentation:
  https://colmap.github.io/

Example usage:
  colmap help [ -h, --help ]
  colmap gui
  colmap gui -h [ --help ]
  colmap automatic_reconstructor -h [ --help ]
  colmap automatic_reconstructor --image_path IMAGES --workspace_path WORKSPACE
  colmap feature_extractor --image_path IMAGES --database_path DATABASE
  colmap exhaustive_matcher --database_path DATABASE
  colmap mapper --image_path IMAGES --database_path DATABASE --output_path MODEL
  ...

Available commands:
  help
  gui
  automatic_reconstructor
  bundle_adjuster
  color_extractor
  database_cleaner
  database_creator
  database_merger
  delaunay_mesher
  exhaustive_matcher
  feature_extractor
  feature_importer
  hierarchical_mapper
  image_deleter
  image_filterer
  image_rectifier
  image_registrator
  image_undistorter
  image_undistorter_standalone
  mapper
  matches_imp

# Process the images/video

This cell runs COLMAP on your uploaded images/videos.

The output is a set of downsampled images, on which the NerF model runs. You are encouraged to see how the output images look.


In [ ]:
scene = 'custom'
!ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/ --sfm-tool colmap


#!ns-process-data video --data /content/data/nerfstudio/custom_data/raw_video/{vid}.mp4 --output-dir /content/data/nerfstudio/custom_data/ --sfm-tool colmap

# scene = 'poster'
# %cd /content/
# !ns-download-data nerfstudio --capture-name=$scene


[20:31:28] 🎉 Done copying images with prefix 'frame_'.                                        ]8;id=984458;file:///usr/local/lib/python3.10/site-packages/nerfstudio/process_data/process_data_utils.py\process_data_utils.py]8;;\:]8;id=210294;file:///usr/local/lib/python3.10/site-packages/nerfstudio/process_data/process_data_utils.py#348\348]8;;\
(    ● ) Copying images...
🌕  Running COLMAP feature extractor...
[20:31:32] 🎉 Done extracting COLMAP features.                                                       ]8;id=146725;file:///usr/local/lib/python3.10/site-packages/nerfstudio/process_data/colmap_utils.py\colmap_utils.py]8;;\:]8;id=44838;file:///usr/local/lib/python3.10/site-packages/nerfstudio/process_data/colmap_utils.py#137\137]8;;\
🏃  Running COLMAP feature matcher...
[20:34:45] 🎉 Done matching COLMAP features.                                                         ]8;id=437396;file:///usr/local/lib/python3.10/site-packages/nerfstudio/process_data/colmap_utils.py\

# Initialize the interactive viewer

We will use an interactive viewer to visualize novel views generated by the NeRF model.

In [ ]:
import os
%cd /content

# Install localtunnel
# We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
!npm install -g localtunnel

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

import time
time.sleep(3) # the previous command needs time to write to url.txt

with open('url.txt') as f:
  lines = f.readlines()
  websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
# from nerfstudio.utils.io import load_from_json
# from pathlib import Path
# json_filename = "nerfstudio/nerfstudio/viewer/app/package.json"
# version = load_from_json(Path(json_filename))["version"]
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)
print("You may need to click Refresh Page after you start training!")
from IPython import display
display.IFrame(src=url, height=800, width="100%")

/content
⠙⠹⠸⠼
changed 22 packages in 671ms
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼https://viewer.nerf.studio/?websocket_url=wss://new-groups-wear.loca.lt
You may need to click Refresh Page after you start training!


# Train the model

You should keep checking the interactive viewer while the model is being trained, to see how the quality of the generated views improves with time.

In [ ]:
!pip install "numpy<2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 105.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.0
    Uninstalling numpy-2.2.0:
      Successfully uninstalled numpy-2.2.0


In [ ]:
%cd /content
import os
import time

model = 'nerfacto'  # ('nerfacto', 'instant-ngp', 'vanilla-nerf')
scene = 'custom'  # Set to "poster" for pre-processed dataset

if os.path.exists(f"data/nerfstudio/custom_data/transforms.json") and scene == 'custom':
    print(f"Training model: {model}")
    start_time = time.time()
    !ns-train {model} --viewer.websocket-port 7007 --viewer.make-share-url True nerfstudio-data --data data/nerfstudio/custom_data --downscale-factor 4
    training_time = time.time() - start_time
    print(f"Training time: {training_time:.2f} seconds")

elif os.path.exists(f"data/nerfstudio/{scene}/transforms.json") and scene == 'poster':
    print(f"Training model: {model} on pre-processed dataset")
    start_time = time.time()
    !ns-train {model} --viewer.websocket-port 7007 --viewer.make-share-url True nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4
    training_time = time.time() - start_time
    print(f"Training time: {training_time:.2f} seconds")
else:
    print('Preprocessing not complete')

流式输出内容被截断，只能显示最后 5000 行内容。
26390 (87.97%)      93.066 ms            5 m, 35 s            48.86 K                                    
26400 (88.00%)      88.516 ms            5 m, 18 s            50.35 K                                    
26410 (88.03%)      90.947 ms            5 m, 26 s            48.95 K                                    
26420 (88.07%)      97.132 ms            5 m, 47 s            46.49 K                                    
26430 (88.10%)      93.464 ms            5 m, 33 s            47.40 K                                    
26440 (88.13%)      92.429 ms            5 m, 29 s            48.05 K                                    
26450 (88.17%)      93.632 ms            5 m, 32 s            48.64 K                                    
26460 (88.20%)      87.140 ms            5 m, 8 s             51.26 K                                    
----------------------------------------------------------------------------------------------------     
Viewer at: http://l

In [ ]:
!ns-eval --load-config outputs/unnamed/instant-ngp/2024-12-16_013904/config.yml

/usr/local/lib/python3.10/site-packages/nerfstudio/field_components/activations.py:32: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float32)
/usr/local/lib/python3.10/site-packages/nerfstudio/field_components/activations.py:39: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, g):
Traceback (most recent call last):
  File "/usr/local/bin/ns-eval", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/site-packages/nerfstudio/scripts/eval.py", line 67, in entrypoint
    tyro.cli(ComputePSNR).main()
  File "/usr/local/lib/python3.10/site-packages/nerfstudio/scripts/eval.py", line 46, in main
    config, pipeline, checkpoint_path, _ = eval_setup(self.load_config)
  File "/usr/local/lib/python3.10/site-packages/nerfstudio/utils/eval

In [ ]:
# @title # Render Video { vertical-output: true }
# @markdown <h3>Export the camera path from within the viewer, then run this cell.</h3>
# @markdown <h5>The rendered video should be at renders/output.mp4!</h5>


base_dir = "/content/outputs/unnamed/nerfacto/"
training_run_dir = base_dir + os.listdir(base_dir)[0]

from IPython.core.display import HTML, display

# display(HTML("<h3>Upload the camera path JSON.</h3>"))
# %cd $training_run_dir
# uploaded = files.upload()
# uploaded_camera_path_filename = list(uploaded.keys())[0]

# config_filename = training_run_dir + "/config.yml"
# camera_path_filename = training_run_dir + "/" + uploaded_camera_path_filename
# camera_path_filename = camera_path_filename.replace(" ", "\\ ").replace("(", "\\(").replace(")", "\\)")

%cd /content/
!ns-render camera-path --load-config outputs/unnamed/nerfacto/2024-12-16_204341/config.yml --camera-path-filename /content/outputs/unnamed/nerfacto/2024-12-16_204341/camera_paths/2024-12-16-20-43-54.json --output-path renders/2024-12-16_204341/2024-12-16-20-43-54.mp4
# !ns-render camera-path --load-config $config_filename --camera-path-filename $camera_path_filename --output-path renders/output.mp4

/content
/usr/local/lib/python3.10/site-packages/nerfstudio/field_components/activations.py:32: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float32)
/usr/local/lib/python3.10/site-packages/nerfstudio/field_components/activations.py:39: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, g):

🏃 🏃 Install tcnn for speedups 🏃 🏃
pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch


🏃 🏃 Install tcnn for speedups 🏃 🏃
pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch


🏃 🏃 Install tcnn for speedups 🏃 🏃
pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch


🏃 🏃 Install tcnn for speedups 🏃 🏃
pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/t

In [ ]:
ns-render camera-path --load-config outputs/unnamed/nerfacto/2024-12-16_204341/config.yml --camera-path-filename /content/outputs/unnamed/nerfacto/2024-12-16_204341/camera_paths/2024-12-16-20-43-54.json --output-path renders/2024-12-16_204341/2024-12-16-20-43-54.mp4

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<ipython-input-13-be8b3205ec3a>, line 1)